In [1]:
import json

# get the station data
with open('data/clean/stations.json', encoding='utf-8') as file:
    file_contents = file.read()

# get the stations of lines
with open('data/clean/LineStations.json', encoding='utf-8') as file:
    line_contents = file.read()
data = json.loads(file_contents)
lines = json.loads(line_contents)

stations = dict()
for i in data:
    stations[str(i['id'])] = i['name']

print(stations)

# make graph
# ['station1', 'station2', 'cost', 'current_line']
graph = []
for i in data:
    for j in i['connections']:
            graph.append([str(i['id']), str(j['id']), j['cost'], j['fromline']])

node_lines = dict()
for i in data:
    node_lines[str(i['id'])] = i['line']
print(graph)
# store the time of changing line in a station
change_lines_time = dict()
for i in data:
    try:
        change_lines_time[i['id']] = i['walkingTime']
    except KeyError:
        pass

# clean the graph and get the nodes
temp = []
temp1 = []
totalcost = 0
for i in graph:
    temp.append(i[0])
    temp1.append(i[1])
nodes = set(temp).union(set(temp1))
print(nodes)

{'56': 'AsiaWorld-Expo', '47': 'Airport', '54': 'Sunny Bay', '55': 'Disneyland Resort', '76': 'Lo Wu', '75': 'Sheung Shui', '74': 'Fanling', '73': 'Tai Wo', '72': 'Tai Po Market', '71': 'University', '69': 'Fo Tan', '68': 'Sha Tin', '67': 'Tai Wai', '8': 'Kowloon Tong', '65': 'Mong Kok East', '64': 'Hung Hom', '94': 'Exhibition Centre', '2': 'Admiralty', '37': 'Chai Wan', '36': 'Heng Fa Chuen', '35': 'Shau Kei Wan', '34': 'Sai Wan Ho', '33': 'Tai Koo', '32': 'Quarry Bay', '31': 'North Point', '30': 'Fortress Hill', '29': 'Tin Hau', '28': 'Causeway Bay', '27': 'Wan Chai', '1': 'Central', '26': 'Sheung Wan', '81': 'Sai Ying Pun', '82': 'HKU', '83': 'Kennedy Town', '49': 'Tiu Keng Leng', '48': 'Yau Tong', '38': 'Lam Tin', '15': 'Kwun Tong', '14': 'Ngau Tau Kok', '13': 'Kowloon Bay', '12': 'Choi Hung', '11': 'Diamond Hill', '10': 'Wong Tai Sin', '9': 'Lok Fu', '7': 'Shek Kip Mei', '16': 'Prince Edward', '6': 'Mong Kok', '5': 'Yau Ma Tei', '84': 'Ho Man Tin', '85': 'Whampo', '103': 'Wu Kai 

In [2]:
# for checking if the line is changed
def checkChangeLine(curLine, nextNode, fromLine):
    if len(curLine) == 0:
        return True
    if fromLine == "Walking":
        return True
    if curLine[-1] not in node_lines[nextNode]:
        return True
    else:
        return False

# for finding the walking time when changing line
def findWalkingTime(curNode, toLine):
    try:
        if toLine == "Walking":
            return 0
        if curNode in change_lines_time:
            for i in change_lines_time[curNode]:
                l = dict(i)
                if l["to"] == toLine:
                    return l['time']
        else:
            return 0
    except Exception:
        return 0

# using dijkstra algorithm for finding the shortest path
def UCS(graph, costs, open, closed, cur_node, totalcosts, line, total_walkTime, stations):
    if cur_node in open:
        open.remove(cur_node)

    closed.add(cur_node)
    for i in graph:
        walktime = 0
        if (i[0] == cur_node and costs[i[0]]+i[2] < costs[i[1]] and i[1] not in closed):
            open.add(i[1])
            extra_cost = 0
            next_line = line[i[0]].copy()
            if (checkChangeLine(line[i[0]], i[1], i[3])):
                next_line.append(i[3])
                walktime = findWalkingTime(i[0], i[3])

            line[i[1]] = next_line
            # print(line[i[1]][-1], i[3])
            # if (line[i[1]][-1] == "Airport Express" and i[3] != "Airport Express"):
            #     print("change to Airport Express")
            #     extra_cost = 100
            # avoid the airport express line

            # add the walking time when changing line
            if walktime is None:
                # store the cost
                costs[i[1]] = costs[i[0]]+i[2]+extra_cost
            else:
                # store the total walking time
                if i[3] == "Walking":
                    total_walkTime[i[1]] = total_walkTime[i[0]] + i[2]
                else:
                    total_walkTime[i[1]] = total_walkTime[i[0]] + walktime
                # store the cost with walking time
                costs[i[1]] = costs[i[0]]+i[2]+extra_cost+walktime

            # store the path
            path[i[1]] = path[i[0]] + ' -> ' + stations[i[1]]

            # store the total cost
            totalcosts[i[1]] = costs[i[1]]-extra_cost

    costs[cur_node] = 999999
    small = min(costs, key=costs.get)
    if small not in closed:
        UCS(graph, costs, open, closed, small, totalcosts, line, total_walkTime, stations)


costs = dict()  # for storing the cost of each node for the program process
path = dict()  # for storing the path
totalcosts = dict()  # for storing the total cost of the path
line = dict()  # for storing the subway line of the path
total_walkTime = dict()  # for storing the total walking time of the path
for i in nodes:
    # costs array contain the distance and the heurisitc of current node
    costs[i] = 999999
    path[i] = ' '
    totalcosts[i] = 999999
    line[i] = []
    total_walkTime[i] = 0
open = set()
closed = set()

# initial the data
start_node_str = input("Enter the Start Station: ")
start_node = list(stations.keys())[list(stations.values()).index(start_node_str)]
open.add(start_node)
path[start_node] = start_node_str
costs[start_node] = 0
totalcosts[start_node] = 0
total_walkTime[start_node] = 0

# program start
UCS(graph, costs, open, closed, start_node, totalcosts, line, total_walkTime, stations)
goal_node_str = input("Enter the Goal Station: ")
goal_node = list(stations.keys())[list(stations.values()).index(goal_node_str)]
# print the result
print("Suggested Route:")
print(path[goal_node])
changeTimes = len(line[goal_node])-1
for i in line[goal_node]:
    if i == "Walking":
        changeTimes -= 1
    if changeTimes < 0:
        changeTimes = 0
print("Interchange times: ", changeTimes)
print("Suggested Line: ", line[goal_node])
print("Least cost is: ", totalcosts[goal_node])
print("Estimate walking time is: ", total_walkTime[goal_node])


Suggested Route:
Tung Chung -> Sunny Bay -> Tsing Yi -> Airport
Interchange times:  1
Suggested Line:  ['Tung Chung Line', 'Airport Express']
Least cost is:  34
Estimate walking time is:  0
